<a href="https://colab.research.google.com/github/kikosikera/2019_08_31_py_pan_np/blob/master/2019_09_06_pd_np_plt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Phase 2 - Final Report 
#### Demographics Analysis
Select Top 5 Cities to launch a New Security System

### Methodology
•	Create a city wise master dataset with attributes like combine crime rate, population growth, housing growth and penetration of top segments

•	Normalize the variables to the same scale and calculate attractiveness score or opportunity for all cities

•	Using the final score, rank the cities and create a combined strategy

### Import Libraries and Datasets

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
from google.colab import drive
drive.mount('/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive/


Download the datasets and read it all into a pandas dataframe.

In [0]:
# survey - Survey Data will have the segment distribution for the variables providing the right attribute which will be used to calculate propensity indices for the segments or markets
df_survey = pd.read_csv("/gdrive/My Drive/Colab Notebooks/Data/PROJ/01_pandas/files/final/data/01_survey_data.csv", sep = ',', low_memory=False)
# segments - Segments and their description
df_segments = pd.read_csv("/gdrive/My Drive/Colab Notebooks/Data/PROJ/01_pandas/files/final/data/02_city_segments.csv", sep = ',', low_memory=False)
# demographics - Market data will have attributes like combine crime rate, population growth, housing growth and penetration of top segments at state city level
df_demographics = pd.read_csv("/gdrive/My Drive/Colab Notebooks/Data/PROJ/01_pandas/files/final/data/03_citywise_demographics.csv", sep = ',', low_memory=False)

In [0]:
# Demographics Master Dataset
df_demographics.shape, list(df_demographics), df_demographics.notnull().sum()

((6759, 28),
 ['Location',
  'State',
  'County',
  'Land Area',
  'WaterArea',
  'Latitude',
  'Longitude',
  'Population Growth Rate',
  'WhitePopulation',
  'BlackPopulation',
  'HispanicPopulation',
  'AsianPopulation',
  'HawaiianPopulation',
  'IndianPopulation',
  'OtherPopulation',
  'MalePopulation',
  'FemalePopulation',
  'Number of Burgalaries reported',
  'MedianAge',
  'Crime Rate',
  'Homicide Reported',
  'CBSA_Type',
  'FinanceNumber',
  'Property Crime Rate',
  'Housing Construction',
  'HouseholdsPerZipCode',
  'AverageHouseValue',
  'Elevation'],
 Location                          6759
 State                             6756
 County                            6756
 Land Area                         6759
 WaterArea                         6759
 Latitude                          6756
 Longitude                         6756
 Population Growth Rate            6718
 WhitePopulation                   6756
 BlackPopulation                   6756
 HispanicPopulation        

In [0]:
df_demographics.head(5)

,Location,State,County,Land Area,WaterArea,Latitude,Longitude,Population Growth Rate,WhitePopulation,BlackPopulation,HispanicPopulation,AsianPopulation,HawaiianPopulation,IndianPopulation,OtherPopulation,MalePopulation,FemalePopulation,Number of Burgalaries reported,MedianAge,Crime Rate,Homicide Reported,CBSA_Type,FinanceNumber,Property Crime Rate,Housing Construction,HouseholdsPerZipCode,AverageHouseValue,Elevation
0,Alabama|Abbeville,AL,Henry,107.7490,3.1280,31.593513,-85.228148,-1.41%,2186.0,1152.0,78.0,18.0,3.0,25.0,45.0,1647.0,1745.0,4.0,46.2,93,NaN,Metro,10020.0,42.5,0.93%,1412.0,88900.0,499.0
1,Alabama|Adamsville,AL,Jefferson,17.2545,0.1725,33.592857,-86.994015,-2.09%,2399.0,1511.0,79.0,17.0,3.0,41.0,45.0,1903.0,2068.0,10.0,39.6,403,NaN,Metro,10050.0,186.5,-0.69%,1534.0,97100.0,600.0
2,Alabama|Addison,AL,Winston,36.8680,0.2910,34.202745,-87.204788,-1.64%,1197.0,2.0,14.0,2.0,0.0,9.0,9.0,613.0,595.0,1.0,40.5,32,NaN,NaN,10060.0,15.5,2.64%,501.0,88700.0,827.0
3,Alabama|Alabaster,AL,Shelby,18.6895,0.3350,33.218709,-86.783546,1.25%,10545.0,1834.0,1338.0,154.0,12.0,108.0,696.0,6443.0,6670.0,19.0,35.6,670,0.5,Metro,10105.0,306,-2.25%,4590.0,160100.0,524.0
4,Alabama|Albertville,AL,Marshall,58.7030,0.5325,34.251568,-86.256980,-0.02%,8165.0,260.0,2794.0,62.0,30.0,181.0,1979.0,5206.0,5207.0,5.0,32.4,779,0.5,Micro,10120.0,378,0.70%,3720.0,101700.0,592.0


In [0]:
# Targeting
# Based on discussions with key stakeholders, 4 Demographics are selected (index=Location)
df_demographics_4 = df_demographics[['Location','Population Growth Rate','Number of Burgalaries reported','Property Crime Rate','Housing Construction']]
df_demographics_4.head(10)

,Location,Population Growth Rate,Number of Burgalaries reported,Property Crime Rate,Housing Construction
0,Alabama|Abbeville,-1.41%,4.0,42.5,0.93%
1,Alabama|Adamsville,-2.09%,10.0,186.5,-0.69%
2,Alabama|Addison,-1.64%,1.0,15.5,2.64%
3,Alabama|Alabaster,1.25%,19.0,306,-2.25%
4,Alabama|Albertville,-0.02%,5.0,378,0.70%
5,Alabama|Alexander City,-0.16%,32.0,358,1.12%
6,Alabama|Aliceville,-0.99%,5.0,21.5,0.95%
7,Alabama|Anniston,-0.49%,207.0,855.5,0.25%
8,Alabama|Arab,-5.22%,15.0,283.5,-3.51%
9,Alabama|Ardmore,0.54%,1.0,11,-0.48%


In [0]:
df_demographics_4.dtypes

Location                           object
Population Growth Rate             object
Number of Burgalaries reported    float64
Property Crime Rate                object
Housing Construction               object
dtype: object

In [0]:
# Cleaning Data
PGR = df_demographics_4['Population Growth Rate'].replace(to_replace ='%', value = '', regex = True).astype(float)
PGR = PGR*(0.01)
NBR = df_demographics_4['Number of Burgalaries reported'].astype(float)
PCR = df_demographics_4['Property Crime Rate'].replace(to_replace =',', value = '.', regex = True).astype(float)
HCO = df_demographics_4['Housing Construction'].replace(to_replace ='%', value = '', regex = True).astype(float)
HCO = HCO*(0.01)

In [0]:
df_demographics_4 = pd.concat([df_demographics_4['Location'], PGR, NBR, PCR, HCO], axis=1)
df_demographics_4.head(10)

,Location,Population Growth Rate,Number of Burgalaries reported,Property Crime Rate,Housing Construction
0,Alabama|Abbeville,-0.0141,4.0,42.5,0.0093
1,Alabama|Adamsville,-0.0209,10.0,186.5,-0.0069
2,Alabama|Addison,-0.0164,1.0,15.5,0.0264
3,Alabama|Alabaster,0.0125,19.0,306.0,-0.0225
4,Alabama|Albertville,-0.0002,5.0,378.0,0.0070
5,Alabama|Alexander City,-0.0016,32.0,358.0,0.0112
6,Alabama|Aliceville,-0.0099,5.0,21.5,0.0095
7,Alabama|Anniston,-0.0049,207.0,855.5,0.0025
8,Alabama|Arab,-0.0522,15.0,283.5,-0.0351
9,Alabama|Ardmore,0.0054,1.0,11.0,-0.0048


In [0]:
df_demographics_4.dtypes

Location                           object
Population Growth Rate            float64
Number of Burgalaries reported    float64
Property Crime Rate               float64
Housing Construction              float64
dtype: object

## Final Deliverable
#### A Master Dataset study will be created combining Top 3 Segments (Phase 1) and Top 4 Demographics

In [0]:
# TOP 3 Segments (Phase 1)
df_segmentation_top3seg = pd.read_csv("/gdrive/My Drive/Colab Notebooks/Data/PROJ/01_pandas/files/final/data/06_segmentation_perc_we.csv", sep = ',', low_memory=False)
df_segmentation_top3seg.sort_values('Total', axis = 0, ascending = False, 
                 inplace = True, na_position ='last')
df_segmentation_top3seg = df_segmentation_top3seg.round(3)
df_segmentation_top3seg.head(3)

,Segments,Household Counts,Wants to buy Security Devices,Will Switch Brands,High & Medium IPA,Pay more to buy Top Quality Electronics,Total
0,Mass Markets,0.036,0.040,0.037,0.001,0.015,0.128
1,Diverse Workers,0.036,0.016,0.039,0.000,0.013,0.104
3,Well-heeled Affluents,0.011,0.058,0.009,0.014,0.005,0.097


In [0]:
df_segmentation_top3seg['Segments'].iloc[0:3]

0             Mass Markets
1          Diverse Workers
3    Well-heeled Affluents
Name: Segments, dtype: object

In [0]:
# Segments by City/State
df_segments.head(5)

,Customer Segment,StateFullName,City,Household Counts
0,Well-heeled Affluents,Pennsylvania,AARONSBURG,0
1,Modest Families,Pennsylvania,AARONSBURG,0
2,Elite Households,Pennsylvania,AARONSBURG,0
3,Mass Markets,Pennsylvania,AARONSBURG,90
4,Young Urban Masses,Pennsylvania,AARONSBURG,1


In [0]:
df_segments.shape, list(df_segments), df_segments.notnull().sum()

((297240, 4),
 ['Customer Segment', 'StateFullName', 'City', 'Household Counts'],
 Customer Segment    297240
 StateFullName       297240
 City                297240
 Household Counts    297240
 dtype: int64)

In [0]:
df_segments_3 = pd.pivot_table(df_segments, values ='Household Counts', index =['City','StateFullName'], 
                         columns = ['Customer Segment'], aggfunc = np.sum)

In [0]:
df_segments_3.head(5)

Customer Segment          Comfortable Retirees  ...  Young Urban Masses
City       StateFullName                        ...                    
AARONSBURG Pennsylvania                      4  ...                   1
ABBEVILLE  Alabama                         585  ...                   5
           Georgia                         198  ...                   4
           Louisiana                       402  ...                 687
           Mississippi                      18  ...                  13

[5 rows x 12 columns]

In [0]:
df_segments_top3 = df_segments_3[['Mass Markets','Diverse Workers','Well-heeled Affluents']]

In [0]:
df_segments_top3.reset_index(inplace = True)

In [0]:
df_segments_top3 = df_segments_top3.rename(columns={'StateFullName': 'State'})
df_segments_top3['City'] = df_segments_top3['City'].str.capitalize()
df_segments_top3['State'] = df_segments_top3['State'].str.capitalize()

In [0]:
df_segments_top3['City'] = df_segments_top3['City'].str.strip()
df_segments_top3['State'] = df_segments_top3['State'].str.strip()

In [0]:
df_segments_top3.head(5), df_segments_top3.shape

(Customer Segment        City  ... Well-heeled Affluents
 0                 Aaronsburg  ...                     0
 1                  Abbeville  ...                     4
 2                  Abbeville  ...                     1
 3                  Abbeville  ...                     7
 4                  Abbeville  ...                     0
 
 [5 rows x 5 columns], (24770, 5))

In [0]:
# Demographics by City/State
df_demographics_4.head(5)

,Location,Population Growth Rate,Number of Burgalaries reported,Property Crime Rate,Housing Construction
0,Alabama|Abbeville,-0.0141,4.0,42.5,0.0093
1,Alabama|Adamsville,-0.0209,10.0,186.5,-0.0069
2,Alabama|Addison,-0.0164,1.0,15.5,0.0264
3,Alabama|Alabaster,0.0125,19.0,306.0,-0.0225
4,Alabama|Albertville,-0.0002,5.0,378.0,0.0070


In [0]:
df_demographics_4_cs = df_demographics_4["Location"].str.split("|", n = 1, expand = True)

In [0]:
df_demographics_4["State"]= df_demographics_4_cs[0]
df_demographics_4["City"]= df_demographics_4_cs[1]

In [0]:
df_demographics_4.drop(columns =["Location"], inplace = True)

In [0]:
df_demographics_4_cs = df_demographics_4[['City','State', 'Population Growth Rate',
 'Number of Burgalaries reported',
 'Property Crime Rate',
 'Housing Construction']]

In [0]:
df_demographics_4_cs['City'] = df_demographics_4_cs['City'].str.capitalize()
df_demographics_4_cs['State'] = df_demographics_4_cs['State'].str.capitalize()

In [0]:
df_demographics_4_cs['City'] = df_demographics_4_cs['City'].str.strip()
df_demographics_4_cs['State'] = df_demographics_4_cs['State'].str.strip()

In [0]:
df_demographics_4_cs.head(5), df_demographics_4_cs.shape

(          City    State  ...  Property Crime Rate  Housing Construction
 0    Abbeville  Alabama  ...                 42.5                0.0093
 1   Adamsville  Alabama  ...                186.5               -0.0069
 2      Addison  Alabama  ...                 15.5                0.0264
 3    Alabaster  Alabama  ...                306.0               -0.0225
 4  Albertville  Alabama  ...                378.0                0.0070
 
 [5 rows x 6 columns], (6759, 6))

In [0]:
# Demographics > Analysis Report >Seven Attributes selected to rank Cities
# Combined Analysis: 4 Demographics and 3 Segments
df_demographics_7 = df_demographics_4_cs.merge(df_segments_top3, how='outer', on=['City','State'])
df_demographics_7.head(10)

,City,State,Population Growth Rate,Number of Burgalaries reported,Property Crime Rate,Housing Construction,Mass Markets,Diverse Workers,Well-heeled Affluents
0,Abbeville,Alabama,-0.0141,4.0,42.5,0.0093,168.0,351.0,4.0
1,Adamsville,Alabama,-0.0209,10.0,186.5,-0.0069,529.0,424.0,5.0
2,Addison,Alabama,-0.0164,1.0,15.5,0.0264,56.0,374.0,1.0
3,Alabaster,Alabama,0.0125,19.0,306.0,-0.0225,826.0,448.0,36.0
4,Albertville,Alabama,-0.0002,5.0,378.0,0.0070,804.0,2350.0,28.0
5,Alexander city,Alabama,-0.0016,32.0,358.0,0.0112,768.0,2001.0,24.0
6,Aliceville,Alabama,-0.0099,5.0,21.5,0.0095,91.0,342.0,6.0
7,Anniston,Alabama,-0.0049,207.0,855.5,0.0025,2242.0,1227.0,253.0
8,Arab,Alabama,-0.0522,15.0,283.5,-0.0351,743.0,518.0,55.0
9,Ardmore,Alabama,0.0054,1.0,11.0,-0.0048,362.0,286.0,6.0


In [0]:
df_demographics_7.shape, list(df_demographics_7), df_demographics_7.notnull().sum()

((24903, 9),
 ['City',
  'State',
  'Population Growth Rate',
  'Number of Burgalaries reported',
  'Property Crime Rate',
  'Housing Construction',
  'Mass Markets',
  'Diverse Workers',
  'Well-heeled Affluents'],
 City                              24903
 State                             24903
 Population Growth Rate             6718
 Number of Burgalaries reported     6726
 Property Crime Rate                6759
 Housing Construction               6718
 Mass Markets                      24770
 Diverse Workers                   24770
 Well-heeled Affluents             24770
 dtype: int64)

In [0]:
df_demographics_7.set_index(["City", "State"], inplace = True, 
                            append = True, drop = True)
df_demographics_7.head(2)

,,,Population Growth Rate,Number of Burgalaries reported,Property Crime Rate,Housing Construction,Mass Markets,Diverse Workers,Well-heeled Affluents
,City,State,,,,,,,
0,Abbeville,Alabama,-0.0141,4.0,42.5,0.0093,168.0,351.0,4.0
1,Adamsville,Alabama,-0.0209,10.0,186.5,-0.0069,529.0,424.0,5.0


In [0]:
# A data normalization is necessary to put all variables to the same scale.
df_demographics_7.loc[:,:]= df_demographics_7.loc[:,:]/df_demographics_7.loc[:,:].sum()
df_demographics_7.round(6).head(3)

,,,Population Growth Rate,Number of Burgalaries reported,Property Crime Rate,Housing Construction,Mass Markets,Diverse Workers,Well-heeled Affluents
,City,State,,,,,,,
0,Abbeville,Alabama,-0.000392,0.000015,0.000018,0.000261,0.000019,0.000039,0.000002
1,Adamsville,Alabama,-0.000582,0.000037,0.000077,-0.000194,0.000059,0.000047,0.000002
2,Addison,Alabama,-0.000456,0.000004,0.000006,0.000741,0.000006,0.000041,0.000000


In [0]:
# Weighted scoring method
# Based on relevance degree, appropriate weights are applied on the  variables to calculate final weighted average scores
df_final_dem_10 = df_demographics_7[['Population Growth Rate','Number of Burgalaries reported','Property Crime Rate','Housing Construction']]*0.10
df_final_seg_10 = df_demographics_7[['Mass Markets','Diverse Workers','Well-heeled Affluents']]*0.20

In [0]:
df_final = pd.concat([df_final_dem_10, df_final_seg_10], axis=1)

In [0]:
df_final = df_final.round(6)
df_final.head(3)

,,,Population Growth Rate,Number of Burgalaries reported,Property Crime Rate,Housing Construction,Mass Markets,Diverse Workers,Well-heeled Affluents
,City,State,,,,,,,
0,Abbeville,Alabama,-0.000039,0.000001,0.000002,0.000026,0.000004,0.000008,0.0
1,Adamsville,Alabama,-0.000058,0.000004,0.000008,-0.000019,0.000012,0.000009,0.0
2,Addison,Alabama,-0.000046,0.000000,0.000001,0.000074,0.000001,0.000008,0.0


In [0]:
df_final.to_csv(r"/gdrive/My Drive/Colab Notebooks/Data/PROJ/01_pandas/files/final/data/07_final_result_.csv", sep = ',', encoding='utf-8')

In [0]:
df_final_result = pd.read_csv("/gdrive/My Drive/Colab Notebooks/Data/PROJ/01_pandas/files/final/data/07_final_result.csv", sep = ',', low_memory=False)

In [0]:
# Using the final weighted score, a new Ranking of cities is created, a combined strategy to identify the TOP 5 Locations/Regions to target
df_final_result['Total'] = df_final_result.sum(axis=1)
df_final_result.sort_values('Total', axis = 0, ascending = False, 
                 inplace = True, na_position ='last')
df_final_result.round(6).head(5)

,City,State,Population Growth Rate,Number of Burgalaries reported,Property Crime Rate,Housing Construction,Mass Markets2,Diverse Workers2,Well-heeled Affluents2,Total
354,Chicago,Illinois,0.0000,0.0023,0.0016,0.0,0.0149,0.003933,0.005667,0.028400
3312,New York,New York,0.0000,0.0029,0.0026,0.0,0.0146,0.001127,0.001478,0.022705
2348,Las Vegas,Nevada,0.0000,0.0026,0.0008,0.0,0.0091,0.003965,0.000778,0.017243
156,Bronx,New York,0.0001,0.0011,0.0015,0.0,0.0090,0.003257,0.002282,0.017239
2566,Los Angeles,California,0.0000,0.0018,0.0016,0.0,0.0090,0.001679,0.002011,0.016090


#### > The TOP 5 CITIES (regions) selected to launch a New Home Security System are:
1. Chicago
2. New York
3. Las Vegas
4. Bronx
5. Los Angeles

In [0]:
%%HTML
<div style="border-right: 600px solid WhiteSmoke;"><div style="border-left: 100px solid WhiteSmoke;"><div style="border-top: 20px solid WhiteSmoke;"><div style="border-bottom: 20px solid WhiteSmoke;"><param name="showShareOptions" value="false" />
<div class='tableauPlaceholder' id='viz1568760016175' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;HF&#47;HF97RKJ5J&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /><param name="showShareOptions" value="false" /><param name='path' value='shared&#47;HF97RKJ5J' /> <param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;HF&#47;HF97RKJ5J&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1568760016175');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='960px';vizElement.style.height='627px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

### Insights Generated

### Top 3 Customer Segments selected
1. HIGH household counts

2. HIGH willingness to buy security devices or switch brands

3. HIGH propensity to pay more to buy quality products


### Top 5 Cities selected

1. HIGH population growth and housing construction

2. HIGH property crime rate or burglaries reported

3. HIGH percentage of top 3 customer segments


In [0]:
%%HTML
<iframe width="1280" height="400" src="https://app.powerbi.com/view?r=eyJrIjoiZTYwOTM3ZmUtZDM4Ni00NmI3LTg0ZGQtMjg3ZTdkOGVlOGY0IiwidCI6ImMzYTZiMzgzLTg0ZjctNDYyYi1hMWEzLTlhODU0M2EzYzMyYSIsImMiOjZ9" frameborder="0" allowFullScreen="true"></iframe>